In [135]:
import gurobipy as gpy
from gurobipy import GRB
from gurobipy import quicksum


In [136]:
# INPUT FORMAT
# 1.  List of node names: [node1, ..., nodeN]
# 2.  (empty line)
# 3.  Distances of node 1 to every "bigger" node: [dist1, ..., distN]
# |   ...
# 3+N.Distances of node N to every "bigger" node: [*, ..., *, distN]

def read_graph(path):
    graph = []
    with open(path, "r", encoding="utf8") as infile:
        node_labels = infile.readline().strip("[]\n\r ").split(", ")
        infile.readline()
        for i, lab in enumerate(node_labels):
            graph.append(list(map(int, infile.readline().replace(
                "*", "0").strip("[]\n\r ").split(", "))))
    print(node_labels)
    print(graph)
    return node_labels, graph


In [137]:
OPT_TYPE = GRB.CONTINUOUS


def get_edges_over_threshold(threshold, labels, graph):
    bad_edges = []
    for i, l in enumerate(graph):
        for j, dist in enumerate(l):
            if dist > threshold:
                bad_edges.append((labels[i],labels[j]))
    print(bad_edges)
    return bad_edges

def create_model_except(labels, graph, exclude=[]):
    model: gpy.Model = gpy.Model("TSP solver")
    variables = {}
    objective = gpy.LinExpr()
    for i, l in enumerate(graph):
        for j, dist in enumerate(l):
            if dist > 0 and (labels[i], labels[j]) not in exclude:
                variables[(i, j)] = model.addVar(
                    0., 1., vtype=OPT_TYPE, name=f"edge{labels[i]}_{labels[j]}")
                objective += dist * variables[(i, j)]
    for i, node in enumerate(labels):
        model.addConstr(quicksum([val for key, val in variables.items(
        ) if i in key]) == 2, name=f"degree{labels[i]}")
    model.setObjective(objective, GRB.MINIMIZE)
    return variables, model


def add_edges(labels, graph, variables, model):
    objective = gpy.LinExpr()
    for i, l in enumerate(graph):
        for j, dist in enumerate(l):
            if dist > 0:
                if (i, j) not in variables:
                    variables[(i, j)] = model.addVar(
                        0., 1., vtype=OPT_TYPE, name=f"edge{labels[i]}_{labels[j]}")
                objective += dist * variables[(i, j)]
    print(objective)
    model.setObjective(objective, GRB.MINIMIZE)
    return variables, model


def get_subtour(subtour, variables):
    linexpr = gpy.LinExpr()
    for edge, var in variables.items():
        if (edge[0] in subtour) == (edge[1] not in subtour):
            linexpr += var
    return linexpr

def get_matching(handle, teeth, variables):
    assert(len(teeth) % 2 == 1)
    linexpr = gpy.LinExpr()
    for edge, var in variables.items():
        if (edge[0] in handle) and (edge[1] in handle):
            linexpr += var
    for tooth in teeth:
        linexpr += variables[tooth]
    return linexpr


In [138]:
def header(texfile):
    texfile.write(r"\vspace{5pt}\\\scalebox{0.6}{" + "\n")
    texfile.write(r"\begin{minipage}{\textwidth}" + "\n")
    texfile.write(r"\centering" + "\n")
    texfile.write(r"\begin{tikzpicture}" + "\n")
def footer(texfile):
    texfile.write(r"\end{tikzpicture}" + "\n")
    texfile.write(r"\end{minipage}" + "\n")
    texfile.write(r"}\vspace{5pt}\\" + "\n")

def extract_sol(val, suffix=""):
    with open(f"sol_val{suffix}.tex","w+",encoding="utf8") as texout:
        texout.write(f"${val:.0f}$")


def convert_subtour(subtours,suffix=""):
    with open(f"subtour_constr{suffix}.tex","w+",encoding="utf8") as texout:
        texout.write(r"\begin{align*}" + "\n")
        texout.write(r"\sum_{e \in \delta(S)} x_e &\geq 2, & \forall S &\in \{" + "\n")
        for tour in subtours:
            tstring = r"\{\text{" + r"},\text{".join(tour) +r"}\}"
            texout.write(fr"{tstring}," + "\n")
        texout.write(r"\}\end{align*}" + "\n")

def convert_matching(matchings,suffix=""):
    with open(f"2match_constr{suffix}.tex","w+",encoding="utf8") as texout:
        texout.write(r"\begin{align*}" + "\n")
        for handle, teeth in matchings:
            hstring = r"\{\text{" + r"},\text{".join(handle) +r"}\}"
            tstring = r"\{" + r",".join([fr'\{{\text{{{x[0]}}},\text{{{x[1]}}}\}}' for x in teeth]) +r"\}"

            texout.write(fr"x(E(H)) + x(D) &\leq {len(handle) + len(teeth)//2 }, & H= {hstring},\ & D={tstring}\\" + "\n")
        texout.write(r"\end{align*}" + "\n")
        

def convert_graph_to_tikz(labels, positions, graph, exclude=[],suffix=""):
    with open(f"init_out{suffix}.tex","w+",encoding="utf8") as texout:
        header(texout)
        texout.write(r"\begin{scope}[every node/.style = {circle, draw}]" + "\n")
        for node, pos in zip(labels, positions):
            texout.write(fr"\node ({node}) at {pos} {{{node}}};"+ "\n")
        texout.write(r"\end{scope}"+ "\n")
        texout.write(r"\begin{scope}[every edge/.style = {draw, semithick}]"+ "\n")
        for i,el in enumerate(graph):
            for j,dist in enumerate(el):
                if dist > 0 and (labels[i],labels[j]) not in exclude:
                    texout.write(fr"\path ({labels[i]}) edge node[anchor=north] {{{dist}}} ({labels[j]});"+ "\n")
        texout.write(r"\end{scope}"+ "\n")
        footer(texout)
        
def convert_sol_to_tikz(labels, positions, variables, suffix=""):
    with open(f"gurobi_out{suffix}.tex","w+",encoding="utf8") as texout:
        header(texout)
        texout.write(r"\begin{scope}[every node/.style = {circle, draw}]" + "\n")
        for node, pos in zip(labels, positions):
            texout.write(fr"\node ({node}) at {pos} {{{node}}};"+ "\n")
        texout.write(r"\end{scope}"+ "\n")
        texout.write(r"\begin{scope}[every edge/.style = {draw, semithick}]"+ "\n")
        for edge, var in variables.items():
            if var.X > 0:
                texout.write(fr"\path ({labels[edge[0]]}) edge node[anchor=north] {{{var.X:.1f}}} ({labels[edge[1]]});"+ "\n")
        texout.write(r"\end{scope}"+ "\n")
        footer(texout)

In [139]:
THRESHOLD_INST1 = 500
POS_INST1 = [(6, 5), (7, 3), (10, 0.5), (10, 4), (6, 10),
             (0, 0), (4, 5), (1.5, 3), (9, 8), (4, 9)]
nodes, graph = read_graph("Instance_1.txt")
BAD_INST1 = get_edges_over_threshold(THRESHOLD_INST1, nodes, graph)
variables, model = create_model_except(nodes, graph, BAD_INST1)
convert_graph_to_tikz(nodes, POS_INST1, graph, BAD_INST1,  "_i1_1")
model.optimize()
print(model.ObjVal)
convert_sol_to_tikz(nodes, POS_INST1, variables,  "_i1_1")
# add subtours
subtours = [["LS", "OM", "MS", "Al"],
            ["Vb", "Ky", "Tn"],
            ["Au", "Ga", "Fl"]]
for tour in subtours:
    expr = get_subtour([nodes.index(x) for x in tour], variables)
    model.addConstr(expr >= 2)
model.optimize()
print(model.ObjVal)
convert_subtour(subtours,  "_i1_2")
convert_sol_to_tikz(nodes, POS_INST1, variables,  "_i1_2")

# add all edges
variables, model = create_model_except(nodes,graph)
for tour in subtours:
    expr = get_subtour([nodes.index(x) for x in tour], variables)
    model.addConstr(expr >= 2)
model.optimize()
print(model.ObjVal)
convert_sol_to_tikz(nodes, POS_INST1, variables,  "_i1_3")
# # convert_subtour(subtours,  "_i1_3")

# model.optimize()
# print(model.ObjVal)
# convert_subtour(subtours,  "_i1_2")
# convert_sol_to_tikz(nodes, POS_INST1, variables,  "_i1_2")

extract_sol(model.ObjVal,  "_i1")


['Al', 'Au', 'Fl', 'Ga', 'Ky', 'LS', 'MS', 'OM', 'Tn', 'Vb']
[[0, 160, 444, 272, 455, 322, 88, 172, 312, 238], [0, 0, 297, 200, 521, 419, 186, 279, 362, 350], [0, 0, 0, 335, 714, 604, 523, 608, 526, 584], [0, 0, 0, 0, 397, 594, 351, 396, 208, 295], [0, 0, 0, 0, 0, 761, 501, 477, 189, 217], [0, 0, 0, 0, 0, 0, 286, 296, 634, 544], [0, 0, 0, 0, 0, 0, 0, 110, 384, 266], [0, 0, 0, 0, 0, 0, 0, 0, 404, 248], [0, 0, 0, 0, 0, 0, 0, 0, 0, 178], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[('Au', 'Ky'), ('Fl', 'Ky'), ('Fl', 'LS'), ('Fl', 'MS'), ('Fl', 'OM'), ('Fl', 'Tn'), ('Fl', 'Vb'), ('Ga', 'LS'), ('Ky', 'LS'), ('Ky', 'MS'), ('LS', 'Tn'), ('LS', 'Vb')]
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 10 rows, 33 columns and 66 nonzeros
Model fingerprint: 0x2253c0d8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range  

In [140]:
BAD_INST2 = [('b', 'e'), ('e', 'k'), ('g', 'k'), ('h', 'k')]
POS_INST2 = [(0.5, 6), (2, 8), (7.5, 8), (10, 6), (7, 5),
             (3, 5), (0, 0), (2, 2), (5, 2.5), (8, 1.5), (10, 0)]
nodes, graph = read_graph("Instance_2.txt")
variables, model = create_model_except(nodes, graph, BAD_INST2)
convert_graph_to_tikz(nodes, POS_INST2, graph, BAD_INST2, suffix="_i2_1")
model.optimize()
print(model.ObjVal)
convert_sol_to_tikz(nodes, POS_INST2, variables,  "_i2_1")

# add subtours
subtours = [["b", "a", "f", "i", "h", "g"],
            ["c", "d", "e", "j", "k"]]
for tour in subtours:
    expr = get_subtour([nodes.index(x) for x in tour], variables)
    model.addConstr(expr >= 2)
model.optimize()
print(model.ObjVal)
convert_subtour(subtours,  "_i2_2")
convert_sol_to_tikz(nodes, POS_INST2, variables,  "_i2_2")

# add all edges
variables, model = create_model_except(nodes, graph)
for tour in subtours:
    expr = get_subtour([nodes.index(x) for x in tour], variables)
    model.addConstr(expr >= 2)
model.optimize()
print(model.ObjVal)
convert_sol_to_tikz(nodes, POS_INST2, variables,  "_i2_3")

# add 2-matchings
matchings = [(["b", "a", "f"], [("a", "g"),("b","i"),("f","h")]),
             (["h", "g", "k"], [("a", "g"),("f","h"),("d","k")])]
for handle, teeth in matchings:
    expr = get_matching([nodes.index(x) for x in handle], [
                        (nodes.index(x[0]), nodes.index(x[1])) for x in teeth], variables)
    print(expr)
    model.addConstr(expr <= len(handle) + len(teeth)//2)
model.optimize()
print(model.ObjVal)
convert_matching(matchings,  "_i2_4")
convert_sol_to_tikz(nodes, POS_INST2, variables,  "_i2_4")

# add 2-matchings
matchings = [(["b", "c", "e"], [("b", "i"),("e","j"),("c","d")]),
             (["h", "g", "k","i","j"], [("a", "g"),("f","h"),("d","k"),("b","i"),("e","j")])]
for handle, teeth in matchings:
    expr = get_matching([nodes.index(x) for x in handle], [
                        (nodes.index(x[0]), nodes.index(x[1])) for x in teeth], variables)
    print(expr)
    model.addConstr(expr <= len(handle) + len(teeth)//2)
model.optimize()
print(model.ObjVal)
convert_matching(matchings,  "_i2_5")
convert_sol_to_tikz(nodes, POS_INST2, variables,  "_i2_5")
extract_sol(model.ObjVal,  "_i2")




['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
[[0, 34, 0, 0, 0, 26, 22, 0, 0, 0, 0], [0, 0, 43, 0, 36, 33, 0, 0, 20, 0, 0], [0, 0, 0, 24, 28, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 20], [0, 0, 0, 0, 0, 37, 0, 0, 0, 21, 45], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 45, 0, 44, 46], [0, 0, 0, 0, 0, 0, 0, 0, 39, 0, 44], [0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 45], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x3e8a4c9d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 4 rows and 6 columns
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf